In [1]:
import pandas as pd
import os
import plotly.express as px
import numpy as np 
import matplotlib.pyplot as plt
from pycorrcat.pycorrcat import plot_corr, corr_matrix

import seaborn as sns
pd.options.display.max_columns = 99

In [2]:
df = pd.read_csv('../data/pre_training_data.csv', sep = '|')

C:\Users\axelb\AppData\Local\Temp\ipykernel_1356\105131456.py:1: DtypeWarning: Columns (2,9) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('../data/pre_training_data.csv', sep = '|')


In [3]:
df.head()

,Date mutation,Valeur fonciere,Code departement,Code commune,Nombre de lots,Type local,Surface reelle bati,Nombre pieces principales,Nature culture,Nature culture speciale,Surface terrain
0,04/01/2019,37220.0,1,53,1.0,Appartement,20.0,1.0,NaN,NaN,0.0
1,04/01/2019,185100.0,1,53,1.0,Dépendance,0.0,0.0,NaN,NaN,0.0
2,04/01/2019,185100.0,1,53,2.0,Appartement,62.0,3.0,NaN,NaN,0.0
3,08/01/2019,209000.0,1,314,0.0,Maison,90.0,4.0,S,NaN,940.0
4,07/01/2019,134900.0,1,350,0.0,Maison,101.0,5.0,S,NaN,490.0


In [4]:
df.columns

Index(['Date mutation', 'Valeur fonciere', 'Code departement', 'Code commune',
       'Nombre de lots', 'Type local', 'Surface reelle bati',
       'Nombre pieces principales', 'Nature culture',
       'Nature culture speciale', 'Surface terrain'],
      dtype='object')

In [5]:
df['Type local'].value_counts()

Type local
Maison                                      706812
Appartement                                 607602
Dépendance                                  440312
Local industriel. commercial ou assimilé    134674
Name: count, dtype: int64

In [6]:
targets = ['Fonciere Privee','Family Office','Marchand de Bien','Institutionnel','Fond immobilier', 'Agricole']

categories = ['Immeuble pleine propriete','Commerce','Bureau','Hotellerie','Entrepot et logistique', 'Terrain Agricole']

##### Foncière Privée : 50% Immeuble pleine propriété / 5% Commerce / 15% Bureau / 30% Hôtellerie

##### Family Office : 70% Immeuble pleine propriété / 5% Commerce / 5% Bureau

##### Marchand de Bien : 1/6 pour tout

##### Institutionnel : 10% Immeuble pleine propriété / 80% Bureau (Grande superficie) / 10% Entrepot et logistique

##### Fond immobilier : 25% Immeuble pleine propriété / 15% Commerce / 15% Bureau / 20% Hôtellerie / 10% Entrepot et logistique / 5% Terrain Agricole

##### Agricole : 5% Commerce / 20% Entrepot et logistique / 75% Terrain Agricole (Grande superficie)

prc = 50+70+10+25+1/6*100


Immeuble pleine propriété = 50/prc*100 Foncière Privée + 70/prc*100 Family Office + 1/6*100/prc*100 Marchand de Bien + 10/prc*100 Institutionnel + 25/prc*100 Fond immobilier

Commerce = 5/30 Foncière Privée + 5/30 Family Office + 15/30 Fond immobilier + 5/30 Agricole

Bureau = 15/115 Foncière Privée + 5/115 Family Office + 80/115 Institutionnel + 15/115 Fond immobilier

Hotellerie = 60% Foncière Privée + 40% Fond immobilier

Entrepot et logistique = 10/40 Institutionnel + 10/40 Fond immobilier + 20/40 Agricole

Terrain Agricole = 5/80 Fond immobilier + 75/80 Agricole

In [7]:
# COLUMNS TO DUMIES
cols = ['Nature culture', 'Nature culture speciale','Code departement', 'Code commune', 'Type local']

df_dummified = pd.get_dummies(df, columns = cols, drop_first=True)

In [17]:
prc = 50+70+10+25+1/6*100

rules = {
    'Immeuble pleine propriete': {
        'Fonciere Privee': 50 / prc * 100,
        'Family Office': 70 / prc * 100,
        'Marchand de Bien': 1 / 6 * 100 / prc * 100,
        'Institutionnel': 10 / prc * 100,
        'Fond immobilier': 25 / prc * 100
    },
    'Commerce': {
        'Fonciere Privee': 5 / 30,
        'Family Office': 5 / 30,
        'Fond immobilier': 15 / 30,
        'Agricole': 5 / 30
    },
    'Bureau': {
        'Fonciere Privee': 15 / 115,
        'Family Office': 5 / 115,
        'Institutionnel': 80 / 115,
        'Fond immobilier': 15 / 115
    },
    'Hotellerie': {
        'Fonciere Privee': 60,
        'Fond immobilier': 40
    },
    'Entrepot et logistique': {
        'Institutionnel': 10 / 40,
        'Fond immobilier': 10 / 40,
        'Agricole': 20 / 40
    },
    'Terrain Agricole': {
        'Fond immobilier': 5 / 80,
        'Agricole': 75 / 80
    }
}
rules

{'Immeuble pleine propriete': {'Fonciere Privee': 29.126213592233015,
  'Family Office': 40.77669902912622,
  'Marchand de Bien': 9.70873786407767,
  'Institutionnel': 5.825242718446602,
  'Fond immobilier': 14.563106796116507},
 'Commerce': {'Fonciere Privee': 0.16666666666666666,
  'Family Office': 0.16666666666666666,
  'Fond immobilier': 0.5,
  'Agricole': 0.16666666666666666},
 'Bureau': {'Fonciere Privee': 0.13043478260869565,
  'Family Office': 0.043478260869565216,
  'Institutionnel': 0.6956521739130435,
  'Fond immobilier': 0.13043478260869565},
 'Hotellerie': {'Fonciere Privee': 60, 'Fond immobilier': 40},
 'Entrepot et logistique': {'Institutionnel': 0.25,
  'Fond immobilier': 0.25,
  'Agricole': 0.5},
 'Terrain Agricole': {'Fond immobilier': 0.0625, 'Agricole': 0.9375}}

In [22]:
df['Type local'].value_counts()

Type local
Maison                                      706812
Appartement                                 607602
Dépendance                                  440312
Local industriel. commercial ou assimilé    134674
Name: count, dtype: int64